In [2]:
import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers, models

## Download dataset

In [ ]:
data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
with zipfile.ZipFile('horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/horse-or-human')

data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
with zipfile.ZipFile('validation-horse-or-human.zip', 'r') as zip_ref:
    zip_ref.extractall('data/validation-horse-or-human')

# Direktori dataset
TRAINING_DIR = 'data/horse-or-human'
VALIDATION_DIR = 'data/validation-horse-or-human'

## Augmentasi dan normalisasi gambar

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Membangun model CNN

In [ ]:
model = tf.keras.models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # DO NOT CHANGE THIS LINE!
])

c:\Users\dzikr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Compile model

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

## Training model

In [ ]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    verbose=1
)

c:\Users\dzikr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 26s 727ms/step - accuracy: 0.5489 - loss: 0.9849 - val_accuracy: 0.6875 - val_loss: 0.6205
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 13s 394ms/step - accuracy: 0.7720 - loss: 0.5861 - val_accuracy: 0.6406 - val_loss: 1.8655
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 13s 392ms/step - accuracy: 0.8616 - loss: 0.3156 - val_accuracy: 0.6875 - val_loss: 1.6492
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 457ms/step - accuracy: 0.8745 - loss: 0.3250 - val_accuracy: 0.7109 - val_loss: 2.0576
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 13s 401ms/step - accuracy: 0.9006 - loss: 0.2184 - val_accuracy: 0.7500 - val_loss: 1.6654
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 509ms/step - accuracy: 0.9213 - loss: 0.1978 - val_accuracy: 0.8008 - val_loss: 0.7188
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 452ms/step - accuracy: 0.9442 - loss: 0.1481 - val_accuracy: 0.6992 - val_loss: 2.7783
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 453ms/step - accuracy: 0.9390 - loss: 0.1506 - val_accu

## Menyimpan model

In [8]:
model.save("model_05.h5")